In [751]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#from PIL import Image
import pandas as pd # conda install pytables
import os

In [815]:
Data_dir = r'X:\processed\LondonZoo\Jewel-wasp_Ampulex-compressa\DLC_femaleandmale\jwasp_femaleandmale-Sanna-2022-09-12\videos'
test_h5 = "jwaspL_nectar-open-close_nectar2DLC_resnet50_jwasp_femaleandmaleSep12shuffle1_1000000.csv"
ROI_dir = r'X:\processed\LondonZoo\Jewel-wasp_Ampulex-compressa\DLC_femaleandmale\jwasp_femaleandmale-Sanna-2022-09-12\roi-coords'

head = 0
thorax = 1

In [816]:
def read_dlc_csv(file_path):
    """
    read .h5 files and return pandas dataframe and a list of bodyparts
    """
    df = pd.read_csv(file_path)
    
    body_parts = np.array(df)[0]
    body_parts = list(body_parts)
    body_parts.remove('bodyparts')
    
    columns = np.array(df)[1]
    data = np.array(df)[2:]
    new_df = pd.DataFrame(data=data, columns=columns)
    new_df = new_df.drop(['coords'], axis=1)
    
    return new_df, body_parts

In [817]:
dlc_data, bodyparts = read_dlc_csv(os.path.join(Data_dir, test_h5))

C:\Users\Sanna\AppData\Local\Temp\ipykernel_17980\1762987095.py:5: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [818]:
def convert_dataframe_to_array_and_return_column_info(df, bodyparts):
    column_names = [i+e for (i, e) in zip(bodyparts, list(df.columns))]
    column_index = dict([(key, value) for key, value in zip(column_names, range(len(column_names)))])
    return np.array(df).T, column_index

In [819]:
dlc_data_array, column_dict = convert_dataframe_to_array_and_return_column_info(dlc_data, bodyparts)

In [820]:
def read_csv(file_path):
    df = pd.read_csv(file_path)
    df = df.drop(['Unnamed: 0'], axis=1)
    return df

In [821]:
ROI_meta = read_csv(os.path.join(ROI_dir, test_h5))

In [822]:
def inside(x_array, xmin, xmax, y_array, ymin, ymax):
    # where in bounds locations
    x_array = x_array.astype('float64')
    y_array = y_array.astype('float64')
    locations_inbounds = np.where((xmin < x_array) & (x_array < xmax) & (ymin < y_array) & (y_array < ymax))
    return locations_inbounds[0]

In [823]:
enclosureROI_meta = ROI_meta.iloc[0]
xmin = enclosureROI_meta['TLX'] 
xmax = enclosureROI_meta['BRX'] 
ymin = enclosureROI_meta['TLY'] 
ymax = enclosureROI_meta['BRY'] 
inbounds = inside(dlc_data_array[column_dict['thoraxx']], xmin, xmax, dlc_data_array[column_dict['thoraxy']], ymin, ymax)

In [824]:
x_array = inbounds_xcoords = dlc_data_array[column_dict['thoraxx']][inbounds].astype('float64')
y_array = inbounds_ycoords = dlc_data_array[column_dict['thoraxy']][inbounds].astype('float64') 

In [825]:
data_list = []
for i in range(len(ROI_meta['ROI'])-1):
    meta = ROI_meta.iloc[i+1]
    xmin = meta['TLX'] 
    xmax = meta['BRX'] 
    ymin = meta['TLY'] 
    ymax = meta['BRY']
    column_name = meta['ROI']
    in_ROI = inside(inbounds_xcoords, xmin, xmax, inbounds_ycoords, ymin, ymax)
    in_ROI_percentage = len(in_ROI)/len(inbounds)*100
    data_list.append([column_name, in_ROI_percentage])

In [826]:
xminlist=[]
xmaxlist=[]
yminlist=[]
ymaxlist=[]
for i in range(len(ROI_meta['ROI'])-1):
    meta = ROI_meta.iloc[i+1]
    xminlist.append(meta['TLX'])
    xmaxlist.append(meta['BRX'])
    yminlist.append(meta['TLY'])
    ymaxlist.append(meta['BRY'])

In [827]:
processed_data = np.array(data_list)
columns = processed_data.T[0]
data = processed_data.T[1]

In [828]:
processed_data

array([['tube', '0.0'],
       ['water_open', '0.0'],
       ['water_box', '0.0'],
       ['control_box', '0.11855240753526473'],
       ['nectar1_box', '0.013022336993510946'],
       ['nectar2_box', '0.006634020732543312'],
       ['nectar2_open', '0.006634020732543312']], dtype='<U32')

In [ ]:
# need to add validation step: 
    ## % occupation of(anything within enclosure but not tube/water_box/control_box,nectar1_box/nectar2_box + tube + water_box + control_box + nectar1_box + nectar2_box) =1 

    ### np.where(((xminlist[0] < x_array) & (x_array < xmaxlist[0]) & (yminlist[0] < y_array) & (y_array < ymaxlist[0])) & ((xminlist[1] < x_array) & (x_array < xmaxlist[1]) & (yminlist[1] < y_array) & (y_array < ymaxlist[1])))


    ### total_not_in_ROI = np.where((xminlist[0] > x_array) & (x_array > xmaxlist[0]) & (xminlist[1] > x_array) & (x_array > xmaxlist[1]) & (xminlist[2] > x_

    ###total_in_ROI = np.sum(processed_data.T[1].astype('float64'))